## Completion finetuning using unsloth

This notebook makes use of unsloth to finetune a model for a completion task.
In this example we will finetune the llama 3.2 base model to generate ascii art. I would recommend using the unsloth library compared to just using the huggingface library as it requires less memory and is faster.

Adapted from unsloth notebooks, if something is broken check on:
https://unsloth.ai/

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3  peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Load base model

In [4]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-3B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=userdata.get('HF_ACCESS_TOKEN')
)

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
tokenizer.clean_up_tokenization_spaces = False

### Add lora to base model and patch with Unsloth

In [5]:
# More info about parameters: https://huggingface.co/docs/peft/v0.11.0/en/package_reference/lora#peft.LoraConfig
target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"]

# When adding special tokens
train_embeddings = False

if train_embeddings:
  target_modules = target_modules + ["lm_head"]

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of lora matrices according to paper not much loss when set relatively low
    target_modules = target_modules,  # On which modules of the llm the lora weights are used
    lora_alpha = 16, # scales the weights of the adapters (more influence on base model), 16 was recommended on reddit
    lora_dropout = 0, # Default on 0.05 in tutorial but unsloth says 0 is better
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", #"unsloth" for very long context, decreases vram
    random_state = 3407,
    use_rslora = False,  # scales lora_alpha with 1/sqrt(r), huggingface says this works better
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [19]:
import shutil
import os

# Delete datasets cache in Colab
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/datasets"), ignore_errors=True)


In [23]:
import pandas as pd
from datasets import Dataset

# Load the parquet file from Hugging Face
df = pd.read_parquet("hf://datasets/pookie3000/ascii-cats/data/train-00000-of-00001.parquet")

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check column names (should include 'ascii')
print(dataset.column_names)

['ascii', 'creature']


In [24]:
empty_prompt = """
{ascii_art}
"""

# Apply formatting
def formatting_prompts_func_no_prompt(examples):
    ascii_art_samples = examples["ascii"]
    training_prompts = []
    for ascii_art in ascii_art_samples:
        training_prompt = empty_prompt.format(ascii_art=ascii_art) + EOS_TOKEN
        training_prompts.append(training_prompt)
    return { "text": training_prompts }

# Apply the mapping
formatted_dataset = dataset.map(formatting_prompts_func_no_prompt, batched=True)

# Preview sample
print(formatted_dataset[0]["text"])


Map:   0%|          | 0/201 [00:00<?, ? examples/s]


    /\_/\           ___
   = o_o =_______    \ \ 
    __^      __(  \.__) )
(@)<_____>__(_____)____/
<|end_of_text|>


In [25]:
datasett = formatted_dataset
dataset = dataset.map(formatting_prompts_func_no_prompt, batched = True)

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

 ### Visualize dataset

In [26]:
for i, sample in enumerate(dataset):
    print(f"\n------ Sample {i + 1} ----")
    print(sample["text"])
    if i > 2:
      break


------ Sample 1 ----

    /\_/\           ___
   = o_o =_______    \ \ 
    __^      __(  \.__) )
(@)<_____>__(_____)____/
<|end_of_text|>

------ Sample 2 ----

|\---/|
| o_o |
 \_^_/
<|end_of_text|>

------ Sample 3 ----

 |\__/,|   (`\
 |_ _  |.--.) )
 ( T   )     /
(((^_(((/(((_/
<|end_of_text|>

------ Sample 4 ----

   |\---/|
   | ,_, |
    \_`_/-..----.
 ___/ `   ' ,""+ \  
(__...'   __\    |`.___.';
  (_,...'(_,.`__)/'.....+
<|end_of_text|>


In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # process 4 batches before updating parameters (parameter update == step)
        num_train_epochs = 5, # between 1 - 3 to prevent overfitting
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/201 [00:00<?, ? examples/s]

In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 201 | Num Epochs = 5 | Total steps = 130
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,237,063,680 (0.75% trained)


Step,Training Loss
1,3.759900
2,3.533300
3,4.389100
4,3.928000
5,3.531300
6,4.097100
7,3.804700
8,3.712600
9,3.467800
10,3.491900


### inference

In [29]:
from transformers import TextStreamer

def generate_ascii_art(model):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer("", return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationMixin
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationConfig
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100):
        print(token)
        pass

In [31]:
for _ in range(3):
  generate_ascii_art(model)

<|begin_of_text|>
   |\__/,|   (`\
   |o o  |__ _) )
 _.'`--.  `  / -
( `     )  /   |
  `___.'  |    |
        `  |   /
        `/  // //
  (_._.) (_._.)

<|end_of_text|>
tensor([128000,    198,    256,  64696,    565,  35645,     91,    256,  29754,
          5779,    256,    765,     78,    297,    220,    765,    565,  28539,
          1763,    721,   3238,     63,    313,     13,    220,   1595,    220,
           611,  18722,      7,   1595,    257,    883,    220,    611,    256,
          9432,    220,   1595,   6101,   3238,    220,    765,    262,   9432,
           286,   1595,    220,    765,    256,  40081,    286,  38401,    220,
           443,   6611,    220,   5570,   1462,   6266,   5570,   1462,   9456,
        128001], device='cuda:0')
<|begin_of_text|>
  |\__/,|   (`\
  |o o  |__ _) )
_  T  _|  `  / 
((_ `^--' /_<  \
  ¯|   |/   \ \_
  | |  |   |  | ||
  (_| |___|  |_))

<|end_of_text|>
tensor([128000,    198,    220,  64696,    565,  35645,     91,    256,  29754,

## Saving

### Save lora adapter

This is both useful for inference and if you want to load the model again

In [33]:
model.push_to_hub(
    "Thufail/Llama-3.2-3B-ascii-cats-lora",
    tokenizer,
    token = userdata.get('HF_ACCESS_TOKEN')
)

README.md:   0%|          | 0.00/560 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Thufail/Llama-3.2-3B-ascii-cats-lora


### Merge model with lora weights and save to gguf

You can then do inference locally with Ollama or llama.cpp

##### Popular quantization methods

- **q4_k_m**  
  4bit quantization. Low memory. All models you pull with ollama uses this quantization.
- **q8_0**  
  8bit quantization. Medium memory.
- **f16**  
  16 bit quantization. A lot of models are already in 16 bit so then no quantization happens
- **not_quantized**  
  Often same as f16.

In [34]:
model.push_to_hub_gguf(
    "Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF",
    tokenizer,
    quantization_method="q4_k_m",
    token = userdata.get('HF_ACCESS_TOKEN')
)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.54 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 20.18it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF/pytorch_model-00001-of-00002.bin...
Unsloth: Saving Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF into f16 GGUF format.
The output location will be /content/Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/Thufail/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF


### Load model and saved lora adapters
For if you want to continue finetuning or want to do inference using the model in safetensor format.

In [35]:
from transformers import TextStreamer
import re

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Thufail/Llama-3.2-3B-ascii-cats-lora",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=userdata.get('HF_ACCESS_TOKEN')
)


def generate_ascii_art(model):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer("", return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationMixin
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationConfig
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100):
        match1 = re.search(r"<\|begin_of_text\|>\s*(.*?)\s*<\|end_of_text\|>", token, re.DOTALL)
        print(match1)
        pass


==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

In [41]:
import re
def generate_ascii_art(model):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer("", return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationMixin
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationConfig
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100):
        output_text = tokenizer.decode(token[0], skip_special_tokens=False)
        match1 = re.search(r"<\|begin_of_text\|>\s*(.*?)\s*<\|end_of_text\|>", output_text, re.DOTALL)
        print(match1)
        print("\n")
        pass

In [42]:
for _ in range(3):
  generate_ascii_art(model)

<|begin_of_text|>
  |\---/|
  | o_o |
  (  T  )
.^`--^'^.
 `.  ; .'
 | | | | |
((_((|))_))
<|end_of_text|>
None


<|begin_of_text|>
 ,_     _,
  |\\___//|
  |=o   o=|
  \=._Y_.=/
  /`^`^\  \  
  |     |  .
 `'---''  /
  (       )
  (       )
  `-----'
<|end_of_text|>
None


<|begin_of_text|>
  /\ ___ /\
 (  >   <  )
  \  >#<  /
  /       \
 /         \
 |         |
  \       /
  /// /// --
<|end_of_text|>
None


